In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from sklearn import linear_model, datasets

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((15*21,3), np.float32)
objp[:,:2] = np.mgrid[0:21,0:15].T.reshape(-1,2)

In [3]:
objp.shape

(315, 3)

In [4]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('a.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (21,15),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (21,15), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [5]:
print(corners.shape)

(315, 1, 2)


In [6]:
imgpoints
imgp1 = np.array(imgpoints)
imgp1.reshape((315,2))

imagepoints = np.ones((315,3))
imagepoints[:,0] = imgp1[0,:,0,0]
imagepoints[:,1] = imgp1[0,:,0,1]

In [7]:
imagepoints = imagepoints.T

In [8]:
imagepoints[:,:5]

array([[107.56755066, 130.83979797, 154.64964294, 178.43469238,
        202.1711731 ],
       [ 83.28165436,  83.35715485,  83.52548218,  83.59564972,
         84.16691589],
       [  1.        ,   1.        ,   1.        ,   1.        ,
          1.        ]])

In [13]:
objp = objp.T
print("objp.shape",objp.shape)
obj = np.zeros((4,315))
objl = np.ones((1,315))
obj[:2,:] = objp[:2,:]
obj[3,:] = objl
print("obj.shape",obj.shape)

objp.shape (3, 315)
obj.shape (4, 315)


In [10]:
obj[:,:5]

array([[0., 1., 2., 3., 4.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [17]:
imgp = imagepoints

In [18]:
objp[:,:5]

array([[0., 1., 2., 3., 4.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [19]:
imgp[:,:5]

array([[107.56755066, 130.83979797, 154.64964294, 178.43469238,
        202.1711731 ],
       [ 83.28165436,  83.35715485,  83.52548218,  83.59564972,
         84.16691589],
       [  1.        ,   1.        ,   1.        ,   1.        ,
          1.        ]])

We now have the image and object points

Arranging the matrix in "THAT" form

In [20]:
# imgp[:,:6]
# obj[:,:6]
# imgp[0,0]

In [21]:
A = []

In [22]:
for i in range(315):
    A.append(np.array([obj[0,i], obj[1][i], obj[2][i],1,0,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i], (-1)*obj[2][i]*imgp[0,i],(-1)*imgp[0,i]]))
    A.append(np.array([0,0,0,0, obj[0,i], obj[1][i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i], (-1)*obj[2][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [23]:
A

[array([   0.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
          -0.        ,   -0.        ,   -0.        , -107.56755066]),
 array([  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   1.        ,
         -0.        ,  -0.        ,  -0.        , -83.28165436]),
 array([   1.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        -130.83979797,   -0.        ,   -0.        , -130.83979797]),
 array([  0.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ,   1.        ,
        -83.35715485,  -0.        ,  -0.        , -83.35715485]),
 array([   2.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        -309.29928589,   -0.        ,   -0.        , -154.649642

In [24]:
len(A)

630

In [25]:
a = np.array(A)
# d.reshape((12,12))
a.shape

(630, 12)

In [26]:
# a[:6,:12]

In [27]:
ata = np.matmul(a.T,a)
ata.shape

(12, 12)

In [28]:
u1, s1, vh1 = np.linalg.svd(ata, full_matrices=True)

print(u1.shape)
print(s1.shape)
print(vh1.shape)

(12, 12)
(12,)
(12, 12)


In [29]:
s1

array([1.68838325e+10, 1.61390333e+09, 6.01843735e+06, 5.45910196e+04,
       9.93164955e+03, 1.12901814e+03, 1.03217110e+02, 4.54386210e+01,
       4.62742642e-03, 6.03432216e-12, 4.32125968e-16, 0.00000000e+00])

In [31]:
vh1

array([[-1.31789625e-03, -7.02678272e-04, -1.73472348e-18,
        -9.18889693e-05, -7.54711887e-04, -5.68251265e-04,
         0.00000000e+00, -6.07110277e-05,  8.62422139e-01,
         5.02696205e-01,  0.00000000e+00,  5.93417582e-02],
       [-1.24163723e-03,  9.55845569e-04, -1.11022302e-16,
        -1.74223429e-05,  3.32079620e-04,  1.72488078e-03,
         0.00000000e+00,  1.22571600e-04,  5.04482021e-01,
        -8.63202362e-01,  0.00000000e+00, -1.93395097e-02],
       [ 1.16264384e-02,  3.49898299e-03,  1.59049748e-15,
         2.73264691e-03,  2.85512698e-03,  1.03603048e-02,
         0.00000000e+00,  2.25935406e-03,  4.15151027e-02,
         4.66294972e-02,  0.00000000e+00, -9.97911173e-01],
       [-4.25590015e-01, -3.33628072e-01, -3.24487616e-14,
        -3.73589908e-02,  7.37401629e-01,  3.99225372e-01,
         1.69406589e-21,  5.50122524e-02, -5.96890056e-04,
         9.80093793e-04,  0.00000000e+00,  1.69495377e-04],
       [-1.01127190e-01, -7.06709871e-01,  7.1823189

In [ ]:
# orig_candidate = vh1[:,11]

In [34]:
candidate1 = vh1[10,:]

In [36]:
p12 = candidate1
p12

array([ 2.39197977e-12, -6.14400170e-15,  3.65596006e-04,  1.11301777e-11,
       -8.84162637e-16,  2.40227739e-12, -9.99999933e-01,  8.60465402e-12,
       -5.48044687e-17,  1.56915542e-17,  0.00000000e+00,  1.02743968e-13])

In [37]:
p12 = p12.reshape((3,4))
p12

array([[ 2.39197977e-12, -6.14400170e-15,  3.65596006e-04,
         1.11301777e-11],
       [-8.84162637e-16,  2.40227739e-12, -9.99999933e-01,
         8.60465402e-12],
       [-5.48044687e-17,  1.56915542e-17,  0.00000000e+00,
         1.02743968e-13]])

In [47]:
from numpy import linalg as LA

norm = LA.norm(p12,axis=0)
print(norm)
p = p12/norm
p

[2.39197993e-12 2.40228524e-12 1.00000000e+00 1.40688124e-11]


array([[ 9.99999931e-01, -2.55756543e-03,  3.65596006e-04,
         7.91124182e-01],
       [-3.69636310e-04,  9.99996729e-01, -9.99999933e-01,
         6.11611965e-01],
       [-2.29117594e-05,  6.53192799e-06,  0.00000000e+00,
         7.30295951e-03]])

In [48]:
obj.shape

(4, 315)

In [49]:
p.shape

(3, 4)

In [50]:
imagepred1 = np.matmul(p,obj)
imagepred1[:,:5]

array([[0.79112418, 1.79112411, 2.79112404, 3.79112398, 4.79112391],
       [0.61161196, 0.61124233, 0.61087269, 0.61050306, 0.61013342],
       [0.00730296, 0.00728005, 0.00725714, 0.00723422, 0.00721131]])

In [51]:
imagepoints[:,:5]

array([[107.56755066, 130.83979797, 154.64964294, 178.43469238,
        202.1711731 ],
       [ 83.28165436,  83.35715485,  83.52548218,  83.59564972,
         84.16691589],
       [  1.        ,   1.        ,   1.        ,   1.        ,
          1.        ]])

In [52]:
error1 = imagepoints - imagepred1
error1[:,:5]

array([[106.77642648, 129.04867386, 151.8585189 , 174.64356841,
        197.38004919],
       [ 82.67004239,  82.74591252,  82.91460949,  82.98514666,
         83.55678247],
       [  0.99269704,   0.99271995,   0.99274286,   0.99276578,
          0.99278869]])

high error

In [60]:
ptp = np.matmul(p.T,p)
ptpinv = inv(ptp)
left = np.matmul(ptpinv,p.T)
left

array([[ 9.97520447e-01,  1.95312500e-03, -1.07886051e+02],
       [ 0.00000000e+00,  0.00000000e+00,  7.10388212e+01],
       [ 3.90625000e-03, -2.00000000e+00,  1.54581106e+02],
       [ 3.12995911e-03,  9.76562500e-04,  1.36528776e+02]])

In [61]:
_norm = LA.norm(left)
leftN = left/_norm
leftN

array([[ 4.09887222e-03,  8.02550949e-06, -4.43310350e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.91902840e-01],
       [ 1.60510190e-05, -8.21812172e-03,  6.35183173e-01],
       [ 1.28611925e-05,  4.01275475e-06,  5.61005048e-01]])

Predicting co-ordinates of objects

In [62]:
predobj = np.matmul(leftN,imgp)
predobj[1,0:30]

array([0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284,
       0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284,
       0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284,
       0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284,
       0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284,
       0.29190284, 0.29190284, 0.29190284, 0.29190284, 0.29190284])

In [63]:
obj[1,0:30]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Now trying the same things after excluding the Z

In [64]:
objp = objp.T
print(objp.shape)
obj = np.zeros((3,315))
# objl = np.ones((1,315))
obj[:2,:] = objp[:2,:]
# obj[3,:] = objl
print(obj.shape)

(315, 3)


ValueError: could not broadcast input array from shape (2,3) into shape (2,315)

In [65]:
obj[:,:5]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [66]:
obj.shape

(3, 315)

In [67]:
C = []

In [68]:
for i in range(315):
    C.append(np.array([obj[0,i], obj[1][i],1,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i],(-1)*imgp[0,i]]))
    C.append(np.array([0,0,0, obj[0,i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [69]:
C.shape

AttributeError: 'list' object has no attribute 'shape'

In [70]:
len(C)

630

In [71]:
C = np.array(C)
C.shape

(630, 9)

In [72]:
ctc = np.matmul(C.T,C)
# ctcinv = inv(ctc)
# ctcinv.shape

In [73]:
print(ctc.shape)
# print(ctc)

(9, 9)


In [74]:
u, s, vh = np.linalg.svd(ctc, full_matrices=True)

In [75]:
print(u.shape)
print(s.shape)
print(vh.shape)

(9, 9)
(9,)
(9, 9)


In [76]:
s

array([6.60526916e+07, 3.15000000e+02, 4.59981352e+01, 4.13515899e-15,
       1.71889297e-28, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [77]:
vh[:,8]

array([ 9.99997964e-01, -1.86853175e-16, -2.01805286e-03, -2.73857449e-19,
        1.42234614e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])

In [78]:
p = vh[:,8]

In [79]:
p = p.reshape((3,3))
# pinv = inv(p)
# pinv
p

array([[ 9.99997964e-01, -1.86853175e-16, -2.01805286e-03],
       [-2.73857449e-19,  1.42234614e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

We need to normalize the matrix H

In [80]:
from numpy import linalg as LA

norm = LA.norm(p)
print(norm)
p = p/norm
p

1.0


array([[ 9.99997964e-01, -1.86853175e-16, -2.01805286e-03],
       [-2.73857449e-19,  1.42234614e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [81]:
objp[:,:5]

array([[ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 2.,  0.,  0.],
       [ 3.,  0.,  0.],
       [ 4.,  0.,  0.],
       [ 5.,  0.,  0.],
       [ 6.,  0.,  0.],
       [ 7.,  0.,  0.],
       [ 8.,  0.,  0.],
       [ 9.,  0.,  0.],
       [10.,  0.,  0.],
       [11.,  0.,  0.],
       [12.,  0.,  0.],
       [13.,  0.,  0.],
       [14.,  0.,  0.],
       [15.,  0.,  0.],
       [16.,  0.,  0.],
       [17.,  0.,  0.],
       [18.,  0.,  0.],
       [19.,  0.,  0.],
       [20.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  1.,  0.],
       [ 2.,  1.,  0.],
       [ 3.,  1.,  0.],
       [ 4.,  1.,  0.],
       [ 5.,  1.,  0.],
       [ 6.,  1.,  0.],
       [ 7.,  1.,  0.],
       [ 8.,  1.,  0.],
       [ 9.,  1.,  0.],
       [10.,  1.,  0.],
       [11.,  1.,  0.],
       [12.,  1.,  0.],
       [13.,  1.,  0.],
       [14.,  1.,  0.],
       [15.,  1.,  0.],
       [16.,  1.,  0.],
       [17.,  1.,  0.],
       [18.,  1.,  0.],
       [19.,  1.,  0.],
       [20.,  1.

In [82]:
objpt = np.ones((3,315))
objpt[:2,:] = obj[:2,:]
objpt[:,:5]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [83]:
imgpre = np.matmul(p,objp)
imgpre

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 315 is different from 3)

In [84]:
imgp

array([[107.56755066, 130.83979797, 154.64964294, 178.43469238,
        202.1711731 , 225.64862061, 249.3981781 , 273.0770874 ,
        296.48498535, 319.58239746, 343.1633606 , 366.5144043 ,
        389.93441772, 413.60021973, 437.34890747, 461.1373291 ,
        485.06295776, 509.17733765, 533.06195068, 557.16821289,
        580.50585938, 107.33692169, 130.93611145, 154.7782135 ,
        178.62882996, 202.2800293 , 225.58125305, 249.2624054 ,
        272.77856445, 296.31332397, 319.44625854, 342.96151733,
        366.31173706, 389.64541626, 413.41949463, 437.02697754,
        461.16882324, 484.8119812 , 508.94085693, 532.95355225,
        556.81744385, 580.48657227, 107.47132111, 130.70368958,
        154.64413452, 178.45004272, 201.51760864, 225.46687317,
        249.30691528, 272.87945557, 296.28707886, 319.3296814 ,
        342.8117981 , 366.140625  , 389.55459595, 413.24215698,
        437.009552  , 460.9944458 , 484.82711792, 509.11346436,
        532.95703125, 556.80236816, 580.

In [85]:
pinv = inv(p)

norm = LA.norm(pinv)
pinv = pinv/norm
print(norm)
pinv

LinAlgError: Singular matrix

In [86]:
obj[:,:5]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [87]:
obj.shape

(3, 315)

In [88]:
check = np.matmul(pinv,imgp)
check

NameError: name 'pinv' is not defined